In [160]:

with open('input.txt', 'r') as file:
    data = file.read()

grid, directions = data.split('\n\n')

grid = grid.split('\n')
directions = directions.split('\n')
directions = ''.join(directions)
map = dict()

def reset_map():
    global map
    map = dict()

    for y, row in enumerate(grid):
        for x, value in enumerate(row):
            map[(x, y)] = value

In [12]:
def get_coord(x, y, direction, step=1) -> tuple:
    if direction == '^':
        return (x, y - step)
    if direction == 'v':
        return (x, y + step)
    if direction == '<':
        return (x - step, y)
    if direction == '>':
        return (x + step, y)
    

In [75]:
def step(x, y, direction, map) -> tuple[int, int]:
    x_new, y_new = get_coord(x, y, direction)
    v = map[(x_new, y_new)]
    if v == '#':
        return x, y
    elif v == '.':
        map[(x, y)] = '.'
        map[(x_new, y_new)] = '@'
        return x_new, y_new
    elif v == 'O':
        i = 1
        while map[get_coord(x, y, direction, i)] == 'O':
            i += 1
        if map[get_coord(x, y, direction, i)] == '.':
            map[(x, y)] = '.'
            map[(x_new, y_new)] = '@'
            map[get_coord(x, y, direction, i)] = 'O'
            return x_new, y_new
        elif map[get_coord(x, y, direction, i)] == '#':
            return x, y
    else:
        raise ValueError(f'Invalid character: ' + v)


In [76]:
def print_map(map) -> None:
    x_max = max([x for x, y in map.keys()])
    y_max = max([y for x, y in map.keys()])
    for y in range(y_max + 1):
        row = ''
        for x in range(x_max + 1):
            row += map[(x, y)]
        print(row)

def find_start(map) -> tuple:
    for k, v in map.items():
        if v == '@':
            return k
        
def calc_sum_gps(map) -> int:
    gps_sum = 0
    for (x, y), v in map.items():
        if v == 'O':
            gps_sum += 100 * y + x
    return gps_sum

In [84]:
reset_map()
x, y = find_start(map)   

for char in directions:
    x, y = step(x, y, char, map)

calc_sum_gps(map)

1421727

# Part 2

In [111]:
# To get the wider warehouse's map, start with your original map and, for each tile, make the following changes:

#     If the tile is #, the new map contains ## instead.
#     If the tile is O, the new map contains [] instead.
#     If the tile is ., the new map contains .. instead.
#     If the tile is @, the new map contains @. instead.

# So it will only make it wider, not taller.

def extend_map(map):
    new_map = dict()
    for (x, y), v in map.items():
        if v == '#':
            new_map[(x * 2, y)] = '#'
            new_map[(x * 2 + 1, y)] = '#'
        elif v == 'O':
            new_map[(x * 2, y)] = '['
            new_map[(x * 2 + 1, y)] = ']'
        elif v == '.':
            new_map[(x * 2, y)] = '.'
            new_map[(x * 2 + 1, y)] = '.'
        elif v == '@':
            new_map[(x * 2, y)] = '@'
            new_map[(x * 2 + 1, y)] = '.'
    return new_map
    
reset_map()
new_map = extend_map(map)
print_map(new_map)


##############
##......##..##
##..........##
##....[][]@.##
##....[]....##
##..........##
##############


In [124]:
reset_map()
new_map = extend_map(map)
print_map(new_map)

def get_all_box_coords(x_orig, y_orig, direction, map, box_coords: list) -> list[tuple[int, int]]:
    x, y = get_coord(x_orig, y_orig, direction)
    if map[(x, y)] == '[':
        return list(set([
            *get_all_box_coords(x, y, direction, map, box_coords + [(x, y)]),
            *get_all_box_coords(x + 1, y, direction, map, box_coords + [(x + 1, y)])
        ]))
    elif map[(x, y)] == ']':
        return list(set([
            *get_all_box_coords(x - 1, y, direction, map, box_coords + [(x - 1, y)]),
            *get_all_box_coords(x, y, direction, map, box_coords + [(x, y)])
        ]))
    else:
        return box_coords
    
print(get_all_box_coords(6, 5, '^', new_map, []))

##############
##......##..##
##..........##
##....[][]@.##
##....[]....##
##..........##
##############
[(7, 4), (6, 3), (6, 4), (7, 3)]


In [139]:
import copy

def any_box_vertical_blocked(box_coords: list[tuple[int, int]], map, direction) -> bool:
    for x, y in box_coords:
        x_new, y_new = get_coord(x, y, direction)
        if map[(x_new, y_new)] == '#':
            return True
    return False

def move_box_coords(box_coords, direction, map) -> None:
    map_copy = copy.deepcopy(map)
    for x, y in box_coords:
        map[(x, y)] = '.'

    for x, y in box_coords:
        x_new, y_new = get_coord(x, y, direction)
        if map_copy[(x, y)] == '[':
            map[(x_new, y_new)] = '['
        if map_copy[(x, y)] == ']':
            map[(x_new, y_new)] = ']'



In [153]:
def step_v2(x, y, direction, map) -> tuple[int, int]:
    x_new, y_new = get_coord(x, y, direction)
    v = map[(x_new, y_new)]
    if v == '#':
        return x, y
    
    elif v == '.':
        map[(x, y)] = '.'
        map[(x_new, y_new)] = '@'
        return x_new, y_new
    
    elif (v == '[' and direction == '>') or (v == ']' and direction == '<'):
        i = 1
        while map[get_coord(x, y, direction, i)] in ['[', ']']:
            i += 1
        if map[get_coord(x, y, direction, i)] == '.':
            for j in range(1, i):
                if map[get_coord(x, y, direction, j)] == '[':
                    map[get_coord(x, y, direction, j)] = ']'
                elif map[get_coord(x, y, direction, j)] == ']':
                    map[get_coord(x, y, direction, j)] = '['

            if direction == '<':
                map[(get_coord(x, y, direction, i))] = '['
            elif direction == '>':
                map[(get_coord(x, y, direction, i))] = ']'
            map[(x, y)] = '.'
            map[(x_new, y_new)] = '@'
            return x_new, y_new
        elif map[get_coord(x, y, direction, i)] == '#':
            return x, y
        
    elif direction in ['^', 'v'] and v in ['[', ']']:
        box_coords: list[tuple[int, int]] = get_all_box_coords(x, y, direction, map, [])
        if any_box_vertical_blocked(box_coords, map, direction):
            return x, y
        else:
            move_box_coords(box_coords, direction, map)

            map[(x, y)] = '.'
            map[(x_new, y_new)] = '@'
            return x_new, y_new
        
    else:
        raise ValueError(f'Invalid character: ' + v)


reset_map()
new_map = extend_map(map)
x, y = find_start(new_map)
x, y = step_v2(x, y, '<', new_map)
x, y = step_v2(x, y, '<', new_map)
x, y = step_v2(x, y, '<', new_map)
x, y = step_v2(x, y, '<', new_map)
x, y = step_v2(x, y, '<', new_map)
x, y = step_v2(x, y, 'v', new_map)
x, y = step_v2(x, y, '<', new_map)
x, y = step_v2(x, y, '^', new_map)
x, y = step_v2(x, y, '^', new_map)
x, y = step_v2(x, y, '^', new_map)
x, y = step_v2(x, y, '^', new_map)
x, y = step_v2(x, y, '<', new_map)
x, y = step_v2(x, y, '<', new_map)
x, y = step_v2(x, y, '^', new_map)
x, y = step_v2(x, y, '^', new_map)
x, y = step_v2(x, y, '>', new_map)
x, y = step_v2(x, y, '>', new_map)
x, y = step_v2(x, y, '>', new_map)
print_map(new_map)

##############
##...@[]##..##
##..........##
##[]........##
##..........##
##....[]....##
##############


In [156]:
def calc_sum_gps_v2(map) -> int:
    gps_sum = 0
    for (x, y), v in map.items():
        if v == '[':
            gps_sum += 100 * y + x
    return gps_sum

In [162]:
from tqdm import tqdm

reset_map()
new_map = extend_map(map)
x, y = find_start(new_map)

for char in tqdm(directions, total=len(directions)):
    x, y = step_v2(x, y, char, new_map)

print_map(new_map)
calc_sum_gps_v2(new_map)


100%|██████████| 20000/20000 [00:36<00:00, 553.22it/s] 

####################################################################################################
##......[][]......##......[][]..[]..[]..##[][]..........[][]..[][][][][]##[]..........[]..[]..[][]##
##..##..[][]....##........[]........##........................[]...........[].[][]...[].##....[]..##
######.[].[]..@...............................................[]####........##[]......[][][]....[]##
##[][][]..[]......[]..[].................[]................................[][].......[][]####....##
##[][][].....................[].........######[]..............................[]........##..[][]..##
##..##[][]......##........................[][][]..............................[][]##...[]..[][][].##
##..[]##..........[]..........................####[]............[]....##..[][][]........[]....[]..##
##...........[]...[]..........[]##...............[][][][]..............[].##[].[].......[]...[]...##
##..........[][]........[]......##.........[][]...##..##...[][]...........[].[].........[].

1463160